In [1]:
import numpy as np
from math import pi
from scipy.special import hankel2, jv

# basic periodic scatter information
from novice_stakes.periodic_scatter import Bragg

# complete reflection coefficent calculation modules to check results
from novice_stakes.periodic_scatter import CosineRs, QuadRs

In [2]:
# acoustic parameters
theta_inc = 35. * pi / 180
fc = 500.  # monofrequency source
c = 1500.  # sound speed, m/s
kc = 2 * pi * fc / c

# source and reciever parameters
xsrc = 0
zsrc = -10.
xrcr = 200.
zrcr = -20.

In [3]:
# setup xaxis centered around receiver
decimation = 8  # integration lengths per wavelength
dx = fc / (8 * c)
ilength = 100000

# compute xaxis
numx = int(np.ceil(ilength / dx))
xaxis = np.arange(numx) * dx + (xrcr - ilength / 2)

In [4]:
# Periodic surface formulation
# Sinusidal surface
H = 2.
L = 45.
K = 2 * pi / L

# Periodic length determines the Bragg scatter angles
numeva = 10
bragg = Bragg(L)
qvec = bragg.qvec(theta_inc, numeva, fc)
a0, aq, b0, bq = bragg.bragg_angles(theta_inc, qvec, fc)

# surface specifactions for one period
num_per = int(np.ceil(L / dx))
x_per = np.arange(num_per) * dx

In [5]:
# Use far field approximation for hankel function for scatter pressure integral
ztest = 25.
hexact = hankel2(0,ztest)
happx = np.sqrt(2 / (pi * ztest)) * np.exp(-1j * (ztest - pi / 4))
np.abs(hexact - happx) / np.abs(hexact)

0.004997094116970992

In [6]:
# Assume no structure for source or surface
# recover the image source for a flat surface
dpinc_KA = (kc * np.sin(theta_inc) / 2) \
        * np.exp(-1j * kc * (np.cos(theta_inc) * xaxis + np.sin(theta_inc) * np.abs(zsrc)))

rra = np.sqrt((xrcr - xaxis) ** 2 + zrcr ** 2)
gra = np.sqrt(2 / (pi * kc * rra)) * np.exp(-1j * (kc * rra - pi / 4))

# negative sign is consistant with other integrals that include hankel of 2nd kind
pKA = -np.sum(dpinc_KA * gra) * dx
pimg = -np.exp(-1j * kc * (np.cos(theta_inc) * xrcr + np.sin(theta_inc) * np.abs(zrcr + zsrc)))
np.abs(pKA - pimg) / np.abs(pimg)

0.004015246327051374

In [7]:
# Assume periodic source and surface, flat surface
# source term
projection = b0
KA_per = -2j * projection * np.exp(-1j * b0 * -zsrc)

# receiver term using grating greens function
gra = np.exp(-1j * (bq[:, None] * -zrcr + qvec[:, None] * K * (xrcr - x_per))) / bq[:, None]
gra = (1j / (2 * L)) * np.sum(gra, axis=0)

# surface integral for scattered pressure
p_sca_per = -np.exp(-1j * a0 * xrcr) * np.sum(KA_per * gra) * dx
np.abs(p_sca_per - pimg) / np.abs(pimg)

1.5151595604785606e-14

In [8]:
# non-structured KA surface integral for a sinusoidal surface
eta = (H / 2) * np.cos(K * xaxis)
eta_p = -(H * K / 2) * np.sin(K * xaxis)

projection = np.dot(np.array([np.cos(theta_inc), np.sin(theta_inc)]), np.array([-eta_p, np.ones_like(xaxis)]))

dpinc_KA = (kc * projection / 2) \
        * np.exp(-1j * kc * (np.cos(theta_inc) * xaxis + np.sin(theta_inc) * np.abs(eta - zsrc)))

rra = np.sqrt((xrcr - xaxis) ** 2 + (zrcr - eta) ** 2)
gra = np.sqrt(2 / (pi * kc * rra)) * np.exp(-1j * (kc * rra - pi / 4))

# negative sign is consistant with other integrals that include hankel of 2nd kind
pKA = -np.sum(dpinc_KA * gra) * dx
pKA

(-0.5768196671478311+0.3048791829637074j)

In [9]:
# Integrate KA using periodic greens function, sinusoidal surface
eta = (H / 2) * np.cos(K * x_per)
eta_p = -(H * K / 2) * np.sin(K * x_per)

# source term
projection = np.dot(np.array([a0, b0]),
                    np.array([-eta_p, np.ones_like(x_per)]))
KA_per = -2j * projection * np.exp(-1j * b0 * (eta - zsrc))

# receiver term
phase = bq[:, None] * (eta - zrcr) + qvec[:, None] * K * (xrcr - x_per)
gra = np.exp(-1j * phase) / bq[:, None]
gra = (1j / (2 * L)) * np.sum(gra, axis=0)

# surface integral for scattered pressure
p_sca_per = -np.exp(-1j * a0 * xrcr) * np.sum(KA_per * gra) * dx
p_sca_per

(-0.5771804077994217+0.30073714779978467j)

In [10]:
# Reflection coefficent formulation for scatter pressure
# source term
projection = np.dot(np.array([a0, b0]),
                    np.array([-eta_p, np.ones_like(x_per)]))

KA_per = -2j * projection * np.exp(-1j * b0 * eta)

# receiver term
gra = (1j / (2 * L)) * np.exp(-1j * (bq[:, None] * eta - qvec[:, None] * K * x_per)) / bq[:, None]

# integration for reflection coefficents
R_int = -np.sum(KA_per * gra, axis=1) * dx

p_sca_r = np.dot(R_int, np.exp(-1j * (-b0 * zsrc + aq * xrcr - bq * zrcr)))
np.abs(p_sca_r - p_sca_per)

2.0513004100461377e-14

In [11]:
# Analytic integration for KA reflection coefficents specific to a sinusoidal surface
r_analytic = 1j ** qvec * jv(qvec, -H * (b0 + bq) / 2) \
           * (a0 * qvec * K / (bq * (b0 + bq)) - b0 / bq)
np.max(np.abs(R_int - r_analytic))

4.825179074965119e-16

In [12]:
# confirm agreement with module calculations
r_cos = CosineRs(H, L, c=c)
r_KA_ana = r_cos.ka(theta_inc, qvec, fc)
p_KA_ana = bragg.p_sca(theta_inc, qvec, fc, r_KA_ana, xsrc, zsrc, xrcr, zrcr)
np.abs(p_sca_r - p_KA_ana)

0.0

In [13]:
# confirm agreement with module calculations
r_quad = QuadRs(x_per, eta, eta_p, c=c)
r_KA_quad = r_quad.ka(theta_inc, qvec, fc)
p_KA_quad = bragg.p_sca(theta_inc, qvec, fc, r_KA_quad, xsrc, zsrc, xrcr, zrcr)
np.abs(p_sca_r - p_KA_quad)

0.0

In [14]:
np.max(np.abs(r_KA_ana - r_KA_quad))

4.825179074965119e-16